<a href="https://colab.research.google.com/github/Flychuban/Space-Soup/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/suno-ai/bark.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 16.2 MB/s eta 0:00:00


In [3]:
import requests
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from bs4 import BeautifulSoup
import re

In [4]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [5]:
def search_urls_space_news():
  #URL = "https://www.google.com/search?q=nasa+news&tbm=nws"
  URL = "https://www.google.com/search?sca_esv=568425080&rlz=1C1CHZN_enBG975BG975&sxsrf=AM9HkKmWvQEyiFIWj11sb6Ld6OCoRQ9Kaw:1695711800085&q=space.com+latest+articles&tbm=nws&source=lnms&sa=X&ved=2ahUKEwi_y_Db2seBAxXkRvEDHZ0BDmIQ0pQJegQIDBAB&biw=1536&bih=747&dpr=1.25"
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [6]:
hrefs = search_urls_space_news()

In [7]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [8]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [9]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.biospace.com/article/releases/abyrx-receives-fda-clearance-for-montage-settable-bone-putty-for-use-in-cardiothoracic-surgery/',
 'https://www.space.com/nasa-may-have-unknowingly-found-and-killed-alien-life-on-mars-50-years-ago-scientist-claims',
 'https://www.space.com/spacex-crew-6-astronauts-international-space-station-farewell',
 'https://www.biospace.com/article/releases/galmed-announces-allowance-of-new-patent-for-aramchol-for-the-treatment-of-pulmonary-and-dermal-fibrosis/',
 'https://www.space.com/autumnal-equinox-fall-begins-september-2023',
 'https://www.space.com/spacex-starlink-launch-group-7-3',
 'https://www.space.com/european-space-agency-freedom-of-information',
 'https://www.biospace.com/article/releases/senseonics-announces-the-last-patient-completion-of-the-365-day-enhance-pivotal-clinical-study/',
 'https://www.space.com/nasa-citizen-scientists-april-2024-total-solar-eclipse',
 'https://www.google.com/search?sca_esv%3D568425080%26rlz%3D1C1CHZN_enBG975BG

In [10]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [11]:
titles, articles = get_text_from_urls(cleaned_urls)

In [12]:
articles[0]

'Skip to main content \nPublished: Sep 26, 2023\n\n First-Ever FDA Clearance for Use of a Settable Bone Putty on the Sternum Delivers Long-awaited Innovation for Enhancing Sternal Repair in Cardiothoracic Surgeries STAMFORD, Conn., Sept. 26, 2023 /PRNewswire/ -- Abyrx , Inc. (http://abyrx.com), a leading biomaterial sciences company with a focus on therapeutic technologies for use during surgical procedures, today announced that the United States Food and Drug Administration (FDA) has provided regulatory clearance for the company\'s MONTAGE® Settable Bone Putty for use in cardiothoracic surgery following sternotomy. This represents the first time that the FDA has granted clearance for the use of a settable bone putty on the sternum, and it follows previous FDA clearances for MONTAGE as a bone void filler, cranial cement, and hemostatic bone putty.   By expanding the range of FDA-cleared surgical applications for MONTAGE, this latest milestone positions Abyrx to build upon the more than

Split text from the article into chunks

In [13]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))
sentences[-1]

['When you purchase through links on our site, we may earn an affiliate commission.',
 ' Here’s how it works.',
 ' Adrift in the Kuiper Belt?',
 ' For 17 years, the New Horizons spacecraft has hurtled at unprecedented speed through the solar system.',
 ' Launched in 2006, it flew past Pluto in July 2015, returning the first close-up images of the planet and its moon.',
 ' Then, in 2019, the probe reconnoitered the Kuiper Belt Object (KBO) Arrokoth.',
 ' Both encounters returned astonishing images and a treasure trove of transformative data.',
 ' With its budget being trimmed for 2024, NASA is making some weighty decisions.',
 '.',
 '.',
 ' and one includes drastically trimming New Horizons funds by replacing the current science staff with a new team in an effort to save about $3 million—a rounding error in terms of the planetary science budget.',
 '\xa0 The principal investigator of the mission from its earliest days, Dr.',
 ' Alan Stern, is not happy about the situation.',
 ' "New Hor

In [14]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[i][current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0
0
0


In [15]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [16]:
joined_chunks[0][1]

' \xa0The results, which were published in the world\'s highest impact cardiothoracic peer-reviewed publication, The Journal of Thoracic and Cardiovascular Surgery, showed that patients who received MONTAGE with wire cerclage had improved physical functional recovery, reduced pain, and shortened hospital stays. 2 Long-term outcomes and sternal integrity were also assessed post-surgery using rigorous patient follow-up and radiographic imaging. \xa0 The use of MONTAGE in cardiac surgery will be further validated by Abyrx in a planned multi-center randomized trial in collaboration with Dr.  Fedak. \xa0 John J.  Pacifico, founder and chief executive officer of Abyrx, commented, "When we initially set a course to develop the MONTAGE technology platform, we believed its polymer backbone had unique capabilities that could ultimately support the use of the product as a new standard of care for treating bone defects during surgical procedures. \xa0 Today, we see the product\'s impressive breadt

Summarize all chunks

In [17]:
def summarize_chunks(chunks):
  summarized_texts = []
  for article_chunks in chunks:
    print(len(article_chunks))
    article_text = []
    result = summarizer(article_chunks, max_length=120, min_length=30, do_sample=False)
    article_text = ' '.join([summ['summary_text'] for summ in result])
    summarized_texts.append(article_text)
  return summarized_texts

In [18]:
summarized_texts = summarize_chunks(joined_chunks)

3
3
2
3
3
2
4
2
2
3


In [19]:
summarized_texts[4]

" Autumn officially begins at 2:50 a.m. EDT on Saturday (Sept. 23) The sun is currently migrating south, having spent the previous six months shining directly on the northern half of our planet . At the official start time of autumn, the sun would appear directly overhead from a ship in the Laccadive Sea, positioned on the equator, near Addu City in the Maldives .  At the North Pole, the sun currently is tracing out a 360-degree circle around the entire sky . At the moment of this year's autumnal equinox, it should theoretically disappear completely from view . But twilight illuminates the sky to some extent whenever the sun's upper rim is less than 18 degrees below the horizon .  At the North Pole, the duration of 24-hour darkness lasts almost 11 weeks, not six months . Joe Rao is an 8-time Emmy-nominated meteorologist who served the Putnam Valley region of New York for over 21 years . Watch NASA's OSIRIS-REx asteroid probe approach Earth in free telescope video ."

In [20]:
all_summarized_text = ""
for summarized_text in summarized_texts:
  all_summarized_text += summarized_text

In [21]:
all_summarized_text

' First-Ever FDA Clearance for Use of a Settable Bone Putty on the Sternum Delivers Innovation for Enhancing Sternal Repair in Cardiothoracic Surgeries . MONTAGE is unique as a moldable putty that adheres to bleeding bone and rapidly hardens to achieve bone-like properties after application .  MONTAGE has the potential to eliminate the need or desire for all other settable and non-settable biomaterials used on bone as it is easy to use, can be molded and shaped to conform to varying anatomy, hardens with bone like properties even in the presence of irrigation and bleeding . MonTAGE FastSet and Flowable products may augment provisional hardware and help stabilize bone fragments during the surgical procedure .  Abyx received FDA clearance for the use of bone putty for use in cardiothoracic surgery . Bone putty is a hardening putty that hardens sternal closure and accelerates post-surgery recovery . A German astrobiologist says NASA\'s Viking landers may have inadvertently killed life-for

Create a news article with all of the summarized texts

In [22]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [23]:
summarized_texts

[' First-Ever FDA Clearance for Use of a Settable Bone Putty on the Sternum Delivers Innovation for Enhancing Sternal Repair in Cardiothoracic Surgeries . MONTAGE is unique as a moldable putty that adheres to bleeding bone and rapidly hardens to achieve bone-like properties after application .  MONTAGE has the potential to eliminate the need or desire for all other settable and non-settable biomaterials used on bone as it is easy to use, can be molded and shaped to conform to varying anatomy, hardens with bone like properties even in the presence of irrigation and bleeding . MonTAGE FastSet and Flowable products may augment provisional hardware and help stabilize bone fragments during the surgical procedure .  Abyx received FDA clearance for the use of bone putty for use in cardiothoracic surgery . Bone putty is a hardening putty that hardens sternal closure and accelerates post-surgery recovery .',
 ' A German astrobiologist says NASA\'s Viking landers may have inadvertently killed li

In [29]:
def make_news_summary(all_chunks):
  summary_space_news = ""
  for chunk in all_chunks:
    inputs = tokenizer(f"Make this text like it will be used in space news report: {chunk}", return_tensors='pt', truncation=True, padding=True)
    outputs = model.generate(**inputs, max_length=512, min_length=len(inputs['input_ids'][0]), temperature=0.25)

    # Convert the generated output tensor to a string
    summary_text = tokenizer.decode(outputs[0], skip_special_tokens=True, max_length=512)
    print(outputs)
    summary_space_news += summary_text + '\n'
  return summary_space_news

In [30]:
summary_space_news = make_news_summary(summarized_texts)

tensor([[    0,   891,    63,   226,  1204, 13648, 15362,    21,     8,   169,
            13,  6817,   474,    17,    63,    21,   169,    16, 16216,    17,
         21783, 13241,  3730,     3,     5,  4523,    15,   474,    17,    63,
            19,     3,     9,   614,    35,    53,   474,    17,    63,    24,
           614,    35,     7,     3, 13072,   138, 12493,    11, 16845,     7,
           442,    18,  3042,  1304,    63,  3938,     3,     5,   891,    63,
           226,  1204, 13648, 15362,    21,     8,   169,    13,  6817,   474,
            17,    63,    21,   169,    16, 16216,    17, 21783, 13241,  3730,
             3,     5,  4523,    15,   474,    17,    63,    19,     3,     9,
           614,    35,    53,   474,    17,    63,    24,   614,    35,     7,
             3, 13072,   138, 12493,    11, 16845,     7,   442,    18,  3042,
          1304,    63,  3938,     3,     5,   891,    63,   226,  1204, 13648,
         15362,    21,     8,   169,    13,  6817,  

In [31]:
summary_space_news

'Abyx received FDA clearance for the use of bone putty for use in cardiothoracic surgery. Bone putty is a hardening putty that hardens sternal closure and accelerates post-surgery recovery. Abyx received FDA clearance for the use of bone putty for use in cardiothoracic surgery. Bone putty is a hardening putty that hardens sternal closure and accelerates post-surgery recovery. Abyx received FDA clearance for the use of bone putty for use in cardiothoracic surgery. Bone putty is a hardening putty that hardens sternal closure and accelerates post-surgery recovery. Abyx received FDA clearance for the use of bone putty for use in cardiothoracic surgery. Bone putty is a hardening putty that hardens sternal closure and accelerates post-surgery recovery. Abyx received FDA clearance for the use of bone putty for use in cardiothoracic surgery.\nNASA\'s Viking landers may have killed life-forms on Mars. Dirk Schulze-Makuch says the experiments may have "overwhelmed" these potential microbes. Othe